<a href="https://colab.research.google.com/github/sainikhilrai/Deep-Learning/blob/master/stackedAutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras


Using TensorFlow backend.


In [0]:
# OM SRI SAI RAM

# Descr: Basic or experiment code to perform loss reserve prediction

##############################
# Step 1 : Reading Sample Data
##############################

import numpy as np
import random
import sklearn
import os

from sklearn.preprocessing import LabelEncoder

os.environ["KERAS_BACKEND"] = "theano"
np.random.seed(7)
random.seed(7)
accident_years = np.arange(2005,2013,1)
development_years = np.arange(0,8,1)
triangle = np.array(([1232,946,520,722,316,165,48,14],
                   [1469,1201,708,845, 461,235,56,18],
                   [1652,1416,959,954,605,287,69,21],
                   [1831,1634,1124,1087,725,314,79,24],
                   [2074,1919,1330,1240,756,359,91,28],
                   [2434,2263,1661,1540,909,432,109,33],
                   [2810,2108,1544,1565,924,439,111,34],
                   [3072,2614,1785,1810,1069,508,128,39]))


In [3]:
print(' Accident Years',accident_years)
print(' Developement Years',development_years)
print(' Input', triangle)


 Accident Years [2005 2006 2007 2008 2009 2010 2011 2012]
 Developement Years [0 1 2 3 4 5 6 7]
 Input [[1232  946  520  722  316  165   48   14]
 [1469 1201  708  845  461  235   56   18]
 [1652 1416  959  954  605  287   69   21]
 [1831 1634 1124 1087  725  314   79   24]
 [2074 1919 1330 1240  756  359   91   28]
 [2434 2263 1661 1540  909  432  109   33]
 [2810 2108 1544 1565  924  439  111   34]
 [3072 2614 1785 1810 1069  508  128   39]]


In [4]:
# Cum calc
C = np.zeros(shape=(np.shape(triangle)[0],np.shape(triangle)[1]))
for i in range(np.shape(triangle)[0]):
    for j in range(np.shape(triangle)[1]):
        C[i,j] = sum(triangle[i,:j+1])

print('C',C)

C [[ 1232.  2178.  2698.  3420.  3736.  3901.  3949.  3963.]
 [ 1469.  2670.  3378.  4223.  4684.  4919.  4975.  4993.]
 [ 1652.  3068.  4027.  4981.  5586.  5873.  5942.  5963.]
 [ 1831.  3465.  4589.  5676.  6401.  6715.  6794.  6818.]
 [ 2074.  3993.  5323.  6563.  7319.  7678.  7769.  7797.]
 [ 2434.  4697.  6358.  7898.  8807.  9239.  9348.  9381.]
 [ 2810.  4918.  6462.  8027.  8951.  9390.  9501.  9535.]
 [ 3072.  5686.  7471.  9281. 10350. 10858. 10986. 11025.]]


In [5]:
l_encode = LabelEncoder()
l_encode.fit(accident_years)
a_yr = l_encode.transform(accident_years)
l_encode.fit(development_years)
dev_yr = l_encode.transform(development_years)

print(a_yr)
print(dev_yr)


[0 1 2 3 4 5 6 7]
[0 1 2 3 4 5 6 7]


In [0]:
train_data = []
for i in range(np.shape(triangle)[0]):
    for j in range(np.shape(triangle)[1]-i):
        train_data.append([a_yr[i],dev_yr[j],C[i,j]])
        
test_data = []
for i in range(1,np.shape(triangle)[0]):
    for j in range(np.shape(triangle)[1]-i,np.shape(triangle)[1]):
        test_data.append([a_yr[i],dev_yr[j],C[i,j]])

 
#convert trainData and testData into numpyArray
train_data = np.array(train_data)
test_data = np.array(test_data)


In [0]:
##############
# define the loss function
################

def poisson_dev(y_true, y_pred):
    return 2*K.mean(y_pred - y_true -y_true*(K.log(K.clip(y_pred,K.epsilon(),None)) -K.log(K.clip(y_true,K.epsilon(),None))),axis=-1)

In [0]:
################################
# Step 3 : Build Model using Neural Networks
# Note : MSE loss used here
###############################

import keras
import keras.backend as K
from keras import regularizers
from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential
from keras.callbacks import RemoteMonitor
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras import optimizers



In [17]:
###################
# Auto Encoder Network Architecture #
######################

#encoder part
inputData= Input(shape=(2,))
encoded= Dense(10,activation='relu',kernel_initializer='normal')(inputData)
encoded= Dense(5,activation='relu')(encoded)
encoded= Dense(2,activation='relu')(encoded)  #latent space representation

#decoder part
decoded = Dense(5, activation='relu')(encoded)
decoded = Dense(10, activation='relu')(decoded)
decoded = Dense(2, activation='relu')(decoded)

#combine the encoder and the decoder
autoencoder1 = Model(inputData, decoded)

#get only the encoder part
encoderPart1= Model(inputData,encoded)

adam = optimizers.Adam(lr=0.1)
autoencoder1.compile(loss="mse", optimizer='adam',metrics=['accuracy'])


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=0)
filepath="v5.best.hdf5"
checkpointer = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)

#history = autoencoder.fit(x=train_data[:,:2], y=train_data[:,:2], batch_size=1, epochs=300, verbose=1, callbacks=[checkpointer,reduce_lr,early_stop], validation_split=0.3, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
history = autoencoder1.fit(x=train_data[:,:2], y=train_data[:,:2], batch_size=1, epochs=300, verbose=1, callbacks=None, validation_split=0.3, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Train on 25 samples, validate on 11 samples
Epoch 1/300
25/25 [==============================] - 0s 19ms/step - loss: 7.3104 - acc: 0.4000 - val_loss: 13.4448 - val_acc: 0.9091
Epoch 2/300
25/25 [==============================] - 0s 2ms/step - loss: 7.1631 - acc: 0.4000 - val_loss: 12.5443 - val_acc: 0.9091
Epoch 3/300
25/25 [==============================] - 0s 1ms/step - loss: 6.9374 - acc: 0.4000 - val_loss: 10.8883 - val_acc: 0.9091
Epoch 4/300
25/25 [==============================] - 0s 2ms/step - loss: 6.6669 - acc: 0.4000 - val_loss: 8.4563 - val_acc: 0.9091
Epoch 5/300
25/25 [==============================] - 0s 2ms/step - loss: 6.3652 - acc: 0.4000 - val_loss: 6.1775 - val_acc: 0.9091
Epoch 6/300
25/25 [==============================] - 0s 2ms/step - loss: 6.1742 - acc: 0.4000 - val_loss: 4.2413 - val_acc: 0.9091
Epoch 7/300
25/25 [==============================] - 0s 2ms/step - loss: 6.0684 - acc: 0.4000 - val_loss: 3.1368 - val_acc: 0.9091
Epoch 8/300
25/25 [================

In [0]:
#get the encoded representation of the train Data and test Data

train_data_Encoded1= encoderPart1.predict(train_data[:,:2])
test_data_Encoded1= encoderPart1.predict(test_data[:,:2])

In [0]:
#make new train data and test Data
newTrainData1= []
newTestData1= []
for i in range(train_data.shape[0]):
  newTrainData1.append([train_data_Encoded1[i,0],train_data_Encoded1[i,1],train_data[i,2]])
  
for i in range(test_data.shape[0]):
  newTestData1.append([test_data_Encoded1[i,0],test_data_Encoded1[i,1],test_data[i,2]])
  
newTrainData1= np.array(newTrainData1)
newTestData1= np.array(newTestData1)



In [21]:
########################################
# BUILD SECOND AUTOENCODER  
########################################


###################
# Auto Encoder Network Architecture #
######################

#encoder part
inputData2= Input(shape=(2,))
encoded2= Dense(20,activation='relu',kernel_initializer='normal')(inputData2)
encoded2= Dense(10,activation='relu')(encoded2)
encoded2= Dense(2,activation='relu')(encoded2)  #latent space representation

#decoder part
decoded2 = Dense(10, activation='relu')(encoded2)
decoded2 = Dense(20, activation='relu')(decoded2)
decoded2 = Dense(2, activation='relu')(decoded2)

#combine the encoder and the decoder
autoencoder2 = Model(inputData2, decoded2)

#get only the encoder part
encoderPart2= Model(inputData2,encoded2)

adam = optimizers.Adam(lr=0.1)
autoencoder2.compile(loss="mse", optimizer='adam',metrics=['accuracy'])


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=0)
filepath="v5.best.hdf5"
checkpointer = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)

#history = autoencoder.fit(x=train_data[:,:2], y=train_data[:,:2], batch_size=1, epochs=300, verbose=1, callbacks=[checkpointer,reduce_lr,early_stop], validation_split=0.3, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
history = autoencoder2.fit(x=newTrainData1[:,:2], y=newTrainData1[:,:2], batch_size=1, epochs=300, verbose=1, callbacks=None, validation_split=0.3, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

Train on 25 samples, validate on 11 samples
Epoch 1/300
25/25 [==============================] - 1s 27ms/step - loss: 0.6047 - acc: 0.6000 - val_loss: 3.9415 - val_acc: 0.0000e+00
Epoch 2/300
25/25 [==============================] - 0s 2ms/step - loss: 0.6047 - acc: 0.6000 - val_loss: 3.9415 - val_acc: 0.0000e+00
Epoch 3/300
25/25 [==============================] - 0s 2ms/step - loss: 0.6047 - acc: 0.6000 - val_loss: 3.9415 - val_acc: 0.0000e+00
Epoch 4/300
25/25 [==============================] - 0s 2ms/step - loss: 0.6047 - acc: 0.6000 - val_loss: 3.9415 - val_acc: 0.0000e+00
Epoch 5/300
25/25 [==============================] - 0s 2ms/step - loss: 0.6047 - acc: 0.6000 - val_loss: 3.9415 - val_acc: 0.0000e+00
Epoch 6/300
25/25 [==============================] - 0s 1ms/step - loss: 0.6047 - acc: 0.6000 - val_loss: 3.9415 - val_acc: 0.0000e+00
Epoch 7/300
25/25 [==============================] - 0s 2ms/step - loss: 0.6047 - acc: 0.6000 - val_loss: 3.9415 - val_acc: 0.0000e+00
Epoch 8/30

In [0]:
#get the encoded representation of the train Data and test Data

train_data_Encoded2= encoderPart2.predict(newTrainData1[:,:2])
test_data_Encoded2= encoderPart2.predict(newTestData1[:,:2])

In [0]:
#make new train data and test Data
newTrainData2= []
newTestData2= []
for i in range(train_data.shape[0]):
  newTrainData2.append([train_data_Encoded2[i,0],train_data_Encoded2[i,1],train_data[i,2]])
  
for i in range(test_data.shape[0]):
  newTestData2.append([test_data_Encoded2[i,0],test_data_Encoded2[i,1],test_data[i,2]])
  
newTrainData2= np.array(newTrainData2)
newTestData2= np.array(newTestData2)



In [24]:
#build the neural Network for prediction

neuralNetwork = Sequential()
ip_dim = 2
#model.add(Dropout(0.1, input_shape=(ip_dim,))
neuralNetwork.add(Dense(ip_dim, input_dim=ip_dim, kernel_initializer='normal', activation='relu'))
neuralNetwork.add(Dense(20, kernel_initializer='normal', activation='relu'))
neuralNetwork.add(Dropout(0.1))
neuralNetwork.add(Dense(15, kernel_initializer='normal', activation='relu'))
neuralNetwork.add(Dropout(0.1))
neuralNetwork.add(Dense(10, kernel_initializer='normal', activation='relu'))
neuralNetwork.add(Dense(1, kernel_initializer='normal',activation="exponential"))
# Compile model
neuralNetwork.compile(loss=poisson_dev, optimizer='adam',metrics=['accuracy'])



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [25]:
history = neuralNetwork.fit(x=newTrainData2[:,:ip_dim], y=newTrainData2[:,ip_dim], batch_size=1, epochs=500, verbose=1, callbacks=None, validation_split=0.33, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)


Train on 24 samples, validate on 12 samples
Epoch 1/500
24/24 [==============================] - 1s 36ms/step - loss: 52219.2236 - acc: 0.0000e+00 - val_loss: 67522.8141 - val_acc: 0.0000e+00
Epoch 2/500
24/24 [==============================] - 0s 2ms/step - loss: 52055.8255 - acc: 0.0000e+00 - val_loss: 67312.6440 - val_acc: 0.0000e+00
Epoch 3/500
24/24 [==============================] - 0s 2ms/step - loss: 51889.8988 - acc: 0.0000e+00 - val_loss: 67106.8132 - val_acc: 0.0000e+00
Epoch 4/500
24/24 [==============================] - 0s 2ms/step - loss: 51726.6102 - acc: 0.0000e+00 - val_loss: 66900.0378 - val_acc: 0.0000e+00
Epoch 5/500
24/24 [==============================] - 0s 2ms/step - loss: 51560.9406 - acc: 0.0000e+00 - val_loss: 66697.0586 - val_acc: 0.0000e+00
Epoch 6/500
24/24 [==============================] - 0s 2ms/step - loss: 51400.2118 - acc: 0.0000e+00 - val_loss: 66485.8577 - val_acc: 0.0000e+00
Epoch 7/500
24/24 [==============================] - 0s 2ms/step - loss: 

In [26]:
###########################################
# Step 4: Model Prediction
#############################################
out = neuralNetwork.predict(newTestData2[:,:2])
print(out)

[[3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]
 [3600.076]]


In [27]:
#################################
# Step 5 : Reserve calculation
#################################

import matplotlib.pyplot as plt
#print(' Test Data',test_data)
#print('pred', model.predict([2,]))
#print('C',C)
true_reserve = 0
for i in range(1,np.shape(triangle)[0]):
    j = np.shape(triangle)[1]-1-i
    #print(i,j)
    #print('last known',C[i,j])
    #print(' last estimate',C[i,np.shape(triangle)[1]-1])
    true_reserve += (C[i,np.shape(triangle)[1]-1] - C[i,j])
    #print(true_reserve)
print(" True reserve",true_reserve)


 True reserve 17352.0


In [0]:
appended= np.array(list(zip(test_data,np.ravel(out))))
len(appended)

out_dict = {}
for i in range(len(appended)):
    #print(tuple(appended[i,0][:2]))
    #print(appended[i,1])
    out_dict[tuple(appended[i,0][:2])] = appended[i,1]



In [0]:
pred_reserve = 0
final_pred = []
actuals = []
for i in range(1,np.shape(triangle)[0]):
    j = int(np.shape(triangle)[1]-1-i)    
    #print(i,j)
    #print('last known',C[i,j])
    #print(' last pred', out_dict[(i,np.shape(triangle)[1]-1)])
    #print(' last estimate',C[i,np.shape(triangle)[1]-1])
    final_pred.append(out_dict[(i,np.shape(triangle)[1]-1)] - C[i,j])
    actuals.append(C[i,np.shape(triangle)[1]-1] - C[i,j])
    pred_reserve += (out_dict[(i,np.shape(triangle)[1]-1)] - C[i,j])

In [30]:
print(" Predicted reserve",pred_reserve)
print(' Bias',pred_reserve - true_reserve)
print('Bias',(pred_reserve-true_reserve)/true_reserve)


 Predicted reserve -12959.468505859375
 Bias -30311.468505859375
Bias -1.7468573366677833
